## 1. 다운로드 및 데이터 전처리

### Settings

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

Importing the required libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
keras = tf.keras
import tensorflow_datasets as tfds
tf.__version__

'2.3.0'

Check out the details (features, statistics, etc.) of the dataset to be downloaded here: https://www.tensorflow.org/datasets/datasets

In [3]:
# List the available datasets
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'dart',
 'davis',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k

### 플라워 데이터셋 다운로드 및 로드하기 `tfds.load()`

In [4]:
# Since "tf_flowers" doesn't define standard splits, 
# use the subsplit feature to divide it into (train, validation, test)
# with 80%, 10%, 10% of the data respectively.

(raw_train, raw_validation, raw_test), metadata = tfds.load(name="tf_flowers", 
                                                            with_info=True,
                                                            split=('train[:80%]', 'train[80%:90%]', 'train[90%:]'),
# specifying batch_size=-1 will load full dataset in the memory
#                                                             batch_size=-1,
# as_supervised: `bool`, if `True`, the returned `tf.data.Dataset`
# will have a 2-tuple structure `(input, label)`                                                            
                                                            as_supervised=True)

In [5]:
print(raw_train)
print(raw_validation)
print(raw_test)

<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>
<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>
<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>


Set the training input image dimensions

In [6]:
IMG_SIZE = 128
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

Create a function which will format the examples in the training dataset as per out needs. The arguments of the `format_example()` depends on the the parameters passed to `tfds.load()`. If `as_supervised=True` then (image, labels) tuple pair will be downloaded else a single dictionary with keys 'image' and 'label' will be passed.

In [7]:
# 기본적인 데이터 전처리 수행하기

def format_example(image, label):
    print("Format example called!")
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    # Resize the image if required
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Format example called!
Format example called!
Format example called!


### 효율적으로 데이터 증폭시키기
Read more [here](https://stackoverflow.com/questions/55141076/how-to-apply-data-augmentation-in-tensorflow-2-0-after-tfds-load) to know how the map function applies data augmentation

In [8]:
def augment_data(image, label):
  print("Augment data called!")
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_contrast(image, lower=0.0, upper=1.0)
  # Add more augmentation of your choice
  return image, label

train = train.map(augment_data)

Augment data called!


Here we'll shuffle the dataset so that we don't have any ordering bias of examples, and create batches of size 32. Setting a shuffle buffer size as large as the dataset ensures that the data is completely shuffled. `.prefetch()` lets the dataset fetch batches, in the background while the model is training.

Without prefetch, the CPU and the GPU/TPU sit idle much of the time

![alt text](https://www.tensorflow.org/images/datasets_without_pipelining.png)

With prefetch, idle time diminishes significantly

![alt text](https://www.tensorflow.org/images/datasets_with_pipelining.png)

In [ ]:
SHUFFLE_BUFFER_SIZE = 1024
BATCH_SIZE = 32

train = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation = validation.batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)
# (Optional) prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
train = train.prefetch(tf.data.experimental.AUTOTUNE)
print(train)
print(validation)
print(test)

In [ ]:
#print(metadata)

<b> 
1. TFDS Reference: https://www.tensorflow.org/datasets/overview
2. For importing data from custom URL or disk location refer, Loading images using tf.data: https://www.tensorflow.org/alpha/tutorials/load_data/images

### 데이터 시각화하기

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Get the function which converts label indices to string
get_label_name = metadata.features['label'].int2str

Here we fetch the datatset batch by batch and convert it to numpy array just before passing it to plotting function.

In [ ]:
plt.figure(figsize=(12,12)) 
for batch in train.take(1):
    for i in range(9):
        image, label = batch[0][i], batch[1][i]
        plt.subplot(3,3,i+1)
        plt.imshow(image.numpy())
        plt.title(get_label_name(label.numpy()))
        plt.grid(False)
print(image.shape)
print(np.min(image))
print(np.max(image))

## 2.  `tf.keras`로 모델 빌드하기

In [ ]:
from tensorflow import keras
keras.__version__

In [ ]:
# Creating a simple CNN model in keras using functional API
def create_model():
    img_inputs = keras.Input(shape=IMG_SHAPE)
    conv_1 = keras.layers.Conv2D(32, (3, 3), activation='relu')(img_inputs)
    maxpool_1 = keras.layers.MaxPooling2D((2, 2))(conv_1)
    conv_2 = keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool_1)
    maxpool_2 = keras.layers.MaxPooling2D((2, 2))(conv_2)
    conv_3 = keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool_2)
    flatten = keras.layers.Flatten()(conv_3)
    dense_1 = keras.layers.Dense(64, activation='relu')(flatten)
    output = keras.layers.Dense(metadata.features['label'].num_classes, activation='softmax')(dense_1)

    model = keras.Model(inputs=img_inputs, outputs=output)
    
    return model

### Visualizing the model

summary를 통해 model의 정보를 요약하고, plot_model 메서드로 모델의 구조를 시각화할 수 있습니다.

In [ ]:
simple_model = create_model()
simple_model.summary()

In [ ]:
keras.utils.plot_model(simple_model, 'flower_model_with_shape_info.png', show_shapes=True)

### Setting training parameters

We specifiy the directory where tensorboard can save its logs

In [ ]:
import datetime, os

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(log_dir)

We also get values of the number of examples in train, val and test sets.
* `steps_per_epoch`: It is the number of batches to train the model in one epoch. Its calculated by dividing the number of training examples by the size of each batch.
* `validation_steps`: It is the same as `steps_per_epoch` but applies to validation dataset

In [ ]:
num_train, num_val, num_test = (
  metadata.splits['train'].num_examples * weight/10 for weight in [8,1,1]
)

steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = round(num_val)//BATCH_SIZE

print('Number of examples in the train set:', num_train)
print('Number of examples in the validation set:', num_val)
print('Number of examples in the test set:', num_test)

### Compile and train the model

In [ ]:
import os

def train_model(model):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Creating Keras callbacks 
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        'training_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5')
    os.makedirs('training_checkpoints/', exist_ok=True)
    early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(train.repeat(),
              epochs=50, 
              steps_per_epoch=steps_per_epoch,
              validation_data=validation.repeat(),
              validation_steps=validation_steps,
              callbacks=[tensorboard_callback,
                         model_checkpoint_callback,
                         early_stopping_checkpoint])
    
    return history

In [ ]:
history = train_model(simple_model)

Plotting the training and validation metrics returned by the `train_model()` routine. We use matplotlib to plot the graph.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

These graphs give us insights into how well our model has trained. It is necessary to ensure that both the training and validation accuracies increase and losses decrease.
* If the <b>training accuracy is high but the validation accuracy is low</b>, then it's a typical case of overfitting. You may have to increase your training dataset by performing data augmentation or downloading more images from the internet. You can also try out other model architectures which include regularisation techniques like Dropout and BatchNormalisation.
* If on the other hand your <b>training accuracy and validation accuracy both are higher</b> but, your <b>validation accuracy is slightly higher</b> then maybe your validation dataset comprises of ideal images of the given classes. Sometimes using techniques like dropout and BatchNorm add randomness in training, making training more difficult. To a lesser extent, it is also because training metrics report the average for an epoch, while validation metrics are evaluated after the epoch, so validation metrics see a model that has trained slightly longer.

## 3. Using pre-trained model

pre-trained model 사용의 개념은 구글, 아마존 그리고 MS와 같은 기업에서 개발된 모델을 사용하는 것입니다. 그런 모델들은 1000개의 카테고리로 구성된 140만개의 이미지인 ImageNet 데이터셋을 학습했습니다. 이런 모델은 1000개의 오브젝트에 대한 기본적인 특징을 이미 학습했기 때문에 강력한 특징추출 능력을 갖고 있습니다.

In [ ]:
from tensorflow import keras

# Create the base model from the pre-trained model MobileNet V2
base_model = keras.applications.InceptionV3(input_shape=IMG_SHAPE,
# We cannot use the top classification layer of the pre-trained model as it contains 1000 classes.
# It also restricts our input dimensions to that which this model is trained on (default: 299x299)
                                               include_top=False, 
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
keras.utils.plot_model(base_model, 'inception_model_with_shape_info.png', show_shapes=True)

우리가 가져온 네트워크는 128x128x3 사진을 영상특징들로 변환하는 특징추출기(feature extractor)입니다. include_top=False 옵션을 통해, 우리 데이터를 위한 분류기를 추가해 학습할 수 있습니다. 

NOTE: It's important to freeze the convolutional based before we compile and train the model. By freezing (or setting `layer.trainable = False`), we prevent the weights in these layers from being updated during training.

### Adding classification head

예측을 생성하기 위해 특징들의 spatial average를 계산하기 위해 `tf.keras.layers.GlobalAveragePooling2d` 레이어를 사용합니다. GAP 레이어는 특징맵을 벡터로 변환해주고, 그 뒤 FCN을 연결해 분류합니다. 

In [9]:
def build_model():
    
    model = keras.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(metadata.features['label'].num_classes, 
                           activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

inception_model = build_model()
inception_model.summary()

NameError: name 'keras' is not defined

### Train the InceptionV3 model

In [ ]:
# Evaluating model before training (Optional)
loss0, accuracy0 = inception_model.evaluate(validation.repeat(), steps = validation_steps)

In [ ]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(log_dir)

# Creating Keras callbacks 
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'training_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5', period=5)
os.makedirs('training_checkpoints/', exist_ok=True)
early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience=5)



history = inception_model.fit(train.repeat(),
                              epochs=5,
                              steps_per_epoch = steps_per_epoch,
                              validation_data=validation.repeat(), 
                              validation_steps=validation_steps,
                              callbacks=[tensorboard_callback,
                              model_checkpoint_callback,
                              early_stopping_checkpoint])

After training the model for 5 epochs we were able to get ~70% accuracy. We plot the learning curves of the training and validation accuracy / loss.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Save keras model
inception_model.save('inception_v3_128_tf_flowes.h5')

In [ ]:
loaded_model = keras.models.load_model('inception_v3_128_tf_flowes.h5')
loaded_model.evaluate(test)

### Fine tuning the model

In [ ]:
# Un-freeze the top layers of the model
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 249

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
    
# Compile the model using a much-lower training rate.
inception_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
inception_model.summary()

If you trained to convergence earlier, this will get you a few percent more accuracy.

In [ ]:
history_fine = inception_model.fit(train.repeat(), 
                                  steps_per_epoch = steps_per_epoch,
                                  epochs=10, 
                                  initial_epoch = 5,
                                  validation_data=validation.repeat(), 
                                  validation_steps=validation_steps,
                                  callbacks=[tensorboard_callback,
                                  model_checkpoint_callback,
                                  early_stopping_checkpoint])

Note: If the training dataset is fairly small, and is similar to the original datasets that Inception V3 was trained on, so fine-tuning may result in overfitting.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

initial_epochs=5

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1], 
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1], 
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In summary here is what we covered on how to do transfer learning using a pre-trained model to improve accuracy: 
* Using a pre-trained model for <b>feature extraction</b> - when working with a small dataset, it is common to leverage the features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier are updated during training. In this case, the convolutional base extracts all the features associated with each image and we train a classifier that determines, given these set of features to which class it belongs. 
* <b>Fine-tuning</b> a pre-trained model - to further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning. In this case, we tune our weights such that we learn highly specified and high level features specific to our dataset. This only make sense when the training dataset is large and very similar to the orginial dataset that the pre-trained model was trained on.

References: 
1. https://www.tensorflow.org/alpha/tutorials/images/intro_to_cnns
2. https://www.tensorflow.org/alpha/tutorials/images/transfer_learning